In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import time
import re

import pandas as pd
import pprint

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection
db = client.nasa_db
collection = db.news

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')
#print(soup.prettify())

In [5]:
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
#results = soup.find_all('div', class_='image_and_description_container')
results = soup.find_all('div', class_='slide')

# Print formatted version of the soup
print(results)

[<div class="slide">
<div class="image_and_description_container">
<a href="/news/8659/alabama-high-school-student-names-nasas-mars-helicopter/">
<div class="rollover_description">
<div class="rollover_description_inner">
Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.
</div>
<div class="overlay_arrow">
<img alt="More" src="/assets/overlay-arrow.png"/>
</div>
</div>
<img alt="Alabama High School Student Names NASA's Mars Helicopter" class="img-lazy" data-lazy="/system/news_items/list_view_images/8659_1-PIA23883-MAIN-320x240.jpg" src="/assets/loading_320x240.png"/>
</a>
</div>
<div class="content_title">
<a href="/news/8659/alabama-high-school-student-names-nasas-mars-helicopter/">
Alabama High School Student Names NASA's Mars Helicopter
</a>
</div>
</div>, <div class="slide">
<div class="image_and_description_container">
<a href="/news/8645/mars-helicopter-attached-to-nasas-perseverance-ro

In [6]:
# Testing the results in looking at the class
# results are returned as an iterable list
#results = soup.find_all('div', class_='image_and_description_container')
#results_test = soup.find_all('div', class_='content_title')

# Print formatted version of the soup
#print(results_test)

In [7]:




# Loop through returned results
for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find('div', class_='content_title').text

        # Identify and return the description
        article = result.find('div', class_='rollover_description_inner').text


        # Run only if title, price, and link are available
        if (title and article):
        #if (article):
            # Print results
            print('-------------')
            print(title)
            print(article)


            # Dictionary to be inserted as a MongoDB document
            post = {
                'title': title,
                'article': article
            }

            collection.insert_one(post)

    except Exception as e:
        print(e)

-------------


Alabama High School Student Names NASA's Mars Helicopter



Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.

-------------


Mars Helicopter Attached to NASA's Perseverance Rover



The team also fueled the rover's sky crane to get ready for this summer's history-making launch.

-------------


NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes



After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.

-------------


10.9 Million Names Now Aboard NASA's Perseverance Mars Rover



As part of NASA's 'Send Your Name to Mars' campaign, they've been stenciled onto three microchips along with essays from NASA's 'Name the Rover' contest. Next stop: Mars.

-------------


Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover



NASA chose a seventh-grader f

In [55]:
# Display items in MongoDB collection
news_listings = db.news.find()

for listing in news_listings:
    print(listing)

{'_id': ObjectId('5ed6ed1f26367b0553ebbc59'), 'title': "\n\nAlabama High School Student Names NASA's Mars Helicopter\n\n", 'article': "\nVaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.\n"}
{'_id': ObjectId('5ed6ed2026367b0553ebbc5a'), 'title': "\n\nMars Helicopter Attached to NASA's Perseverance Rover\n\n", 'article': "\nThe team also fueled the rover's sky crane to get ready for this summer's history-making launch.\n"}
{'_id': ObjectId('5ed6ed2026367b0553ebbc5b'), 'title': "\n\nNASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes\n\n", 'article': '\nAfter the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.\n'}
{'_id': ObjectId('5ed6ed2026367b0553ebbc5c'), 'title': "\n\n10.9 Million Names Now Aboard NASA's Perseverance Mars Rover\n\n", 'article': "\nAs part of NASA's 'Send Your Name to Mars' 

In [9]:
#try:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#except Exception as e:
#    print(e)

In [10]:
#featured_image_url
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [11]:
# Iterate through all pages
#for x in range(50):
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup_mars = bs(html, 'html.parser')
#print(soup_mars)
# Retrieve all elements that contain book information
articles = soup_mars.find_all('ul', class_='articles')
print(articles)    

[<ul class="articles">
<li class="slide">
<a class="fancybox" data-description="Six views of the Martian moon Phobos captured by NASAs Odyssey orbiter as of March 2020. The orbiters THEMIS camera is used to measure temperature variations that suggest what kind of material the moon is made of." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/largesize/PIA23893_hires.jpg" data-link="/spaceimages/details.php?id=PIA23893" data-thumbnail="/spaceimages/images/wallpaper/PIA23893-640x350.jpg" data-title="Odyssey" of="" phobos'="" s="" six="" views="">
<div class="image_and_description_container">
<div class="rollover_description">
<h3 class="release_date">June 8, 2020</h3>
<div class="item_tease_overlay">Odyssey's Six Views of Phobos</div>
<div class="overlay_arrow">
<img alt="more arrow" src="/assets/images/overlay-arrow.png"/>
</div>
</div>
<div class="img">
<img alt="Odyssey" class="thumb" of="" phobos'="" s="" six="" src="/spaceimages/images/wallpaper/PIA23893-640x350.

In [12]:
url_link = []
image_list = []
collection1 = db.image_mars

# Iterate through each book
for article in articles:
    # Use Beautiful Soup's find() method to navigate and retrieve attributes
    line = article.find('li')
    link = line.find('a')
    href = link['data-fancybox-href']
#        title = link['title']
    print('-----------')
    print(href)
    print('https://www.jpl.nasa.gov' + href)
    full_href = ('https://www.jpl.nasa.gov' + href)
    url_link.append(href)
#    image_list = ['https://www.jpl.nasa.gov' + url for url in url_link]
                # Dictionary to be inserted as a MongoDB document
    post1 = {
        'href': full_href,
    }
            
    collection1.insert_one(post1)

# Click the 'Next' button on each page
try:
    browser.click_link_by_partial_text('next')

except:
    print("Scraping Complete")

-----------
/spaceimages/images/largesize/PIA23893_hires.jpg
https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23893_hires.jpg


C:\Users\saman\anaconda3\envs\myenv2\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


Scraping Complete


In [13]:
#image_list

In [14]:
#https://twitter.com/marswxreport?lang=en



In [15]:
#featured_image_url
url_twitter = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url_twitter)

In [16]:
# Iterate through all pages
for x in range(1):
    # HTML object
    html_twitter = browser.html
    # Parse HTML with Beautiful Soup
    soup_twitter = bs(html_twitter, 'html.parser')
    #print(soup_twitter.prettify())
    # Retrieve all elements that contain book information
    #tweets = soup_twitter.find_all('span')[0]
    tweets = soup_twitter.find_all("span", {"class": re.compile("^css-")})
    #print(tweets)  
    print(tweets)  

[<span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"></span>, <span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Log in</span></span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Log in</span>, <span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"><span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Sign up</span></span>, <span class="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0">Sign up</span>, <span class="css-901oao css-16my406 css-bfa6kz r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0"></span>]


In [17]:
    x=1
    collection2 = db.weather
    for tweet in tweets:
#        msg=(tweet.text)
#            print(msg)
#        try:
        if((tweet.text.find('InSight')==0) and x==1): 
            
 #           print(tweet.text.is_text_present('InSight'))
#            print(tweet.is_text_present('InSight'))
            print(tweet.text, type(tweet.text))
#            print('page:', x, '-------------')
#            print((tweet.text).startswith('InSight'))
            print(tweet.text)
            weather = tweet.text
            time.sleep(3)
            post2 = {
                'weather': weather,
            }

            collection2.insert_one(post2)
#        except tweet.text = '@MarsWxReport':
#            print('NO')
        #collection.insert_one(post)
#        browser.quit()
            x+=1
#    browser.click_link_by_partial_text('Next')
    #except (ElementDoesNotExist):
        

In [18]:
#Review the facts and scrape the mars facts from the page

In [19]:
url_facts = 'https://space-facts.com/mars/'

In [20]:
tables = pd.read_html(url_facts)
pprint.pprint(tables)

[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
0               Diameter:         6,779 km        12,742 km
1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
2                  Moons:                2                1
3      Distance from Sun:   227,943,824 km   149,598,262 km
4         Length of Year:   687 Earth days      365.24 days
5            Temperature:    -153 to 20 °C      -88 to 58°C,
                       0  

In [21]:
type(tables)

list

In [22]:
#Comparision information
df_facts = tables[1]
#df.columns = ['Equatorial Diameter', 'Polar Diameter', 'Mass', 'Moons', 
#              'Orbit Distance', 'Orbit Period', 'Surface Temperature', 'First Record', 
#              'Recorded By']

df_facts.columns = ['Comparision', 'Mars', 'Earth']

df_facts.head()

,Comparision,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days


In [23]:
#Information
df_factd = tables[0]
#df_factd.columns = ['Equatorial Diameter', 'Polar Diameter', 'Mass', 'Moons', 
#              'Orbit Distance', 'Orbit Period', 'Surface Temperature', 'First Record', 
#              'Recorded By']

df_factd.columns = ['data_name', 'mars_data']

df_factd.head(9)

,data_name,mars_data
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [24]:
#make html page
html_fact_table = df_factd.to_html()
html_fact_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>data_name</th>\n      <th>mars_data</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millenniu

In [25]:
#https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars

In [26]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [30]:
url_mars_img = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_mars_img)

In [31]:
html = browser.html
soup_mars_img = bs(html, 'html.parser')

img_links = soup_mars_img.find_all('div', class_='item')
img_links

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiapa

In [32]:
img_link_news = []

for img_link in img_links:
    next_url = img_link.find('a')['href']
#        next_url = link
    print(next_url)
#    url_list.append(book_url)
    print('https://astrogeology.usgs.gov/' + next_url)
    long_next_url = ('https://astrogeology.usgs.gov' + next_url)
    img_link_news.append(long_next_url)

print('---------new link-----------------------')    
print(img_link_news)


/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced
/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced
/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced
/search/map/Mars/Viking/valles_marineris_enhanced
https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced
---------new link-----------------------
['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']


In [57]:
print('-----------begin large image---------------------')
collection2 = db.large_image_mars

for img_link_new in img_link_news:
#    for x in range(1, 1):
        browser.visit(img_link_new)
        browser.click_link_by_partial_text('Sample')
        html = browser.html
#        print(html)
        soup_largeimage = bs(html, 'html.parser')
#        print(soup_largeimage)
        bigger_image = soup_largeimage.find('div', class_='downloads').find('a')['href']
        #.find_by_text('Sample')
        post2 = {
                'href': bigger_image,
            }
            
        collection2.insert_one(post2)

#        browser.click_link_by_partial_text('Sample')
    #    large_image = browser.links.find_by_partial_text('full.jpg')
#        large_image = browser.click_link_by_partial_text('Sample').find('a')[href]
    #    soup_mars_img = bs(html, 'html.parser')
#        print(large_image)
        print(bigger_image)
        print('--------end large image------------------------')
    #    sample_image = large_image.find('img')['src']
    #    print(sample_image)

-----------begin large image---------------------
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
--------end large image------------------------
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
--------end large image------------------------
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
--------end large image------------------------
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
--------end large image------------------------


In [58]:
# Display items in MongoDB collection
large_img_listings = db.large_image_mars.find()

for listing in large_img_listings:
    print(listing)

{'_id': ObjectId('5edf2cc3153b88789e0d249b'), 'href': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
{'_id': ObjectId('5edf2cd2153b88789e0d249c'), 'href': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
{'_id': ObjectId('5edf2cd8153b88789e0d249d'), 'href': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
{'_id': ObjectId('5edf2cde153b88789e0d249e'), 'href': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}
